In [3]:
import os
from dotenv import load_dotenv

import openai

from langchain import LLMChain, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

import re
import json

In [88]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

manager = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0
)

# 0) Subtask Manager

지난 주에 구현했던 subtask manager를 간추려서 만듦

- 이전
  - subtask, sutask_arguments, output_type 등으로 스키마가 복잡하고 manger가 subtask를 일부 실행하는 듯 했음
  - list, schema, demonstration 전달
- 이후
  - 오직 subtask만 추려내고, expert model에 subtask schema가 아닌 user query를 그대로 전달함
  - list, demonstration 전달

In [95]:
## subtask manager

hj_subtask = '''The AI assistant should analyze the dialogue context to detect which sub-task should be selected.

Sub-task name: It provides a unique identifier for sub-task detection, which is used for references to related expert models and their predicted results.
Thess are the subtask lists. [ User preference elicitation, Recommendation, Explanation, Item detail search ]

There are some cases for your reference.

user_query: I want to find a legal drama
answer: user preference elicitation

user_query: I want the female actor as the lead role
answer: recommendation

user_query: Why do you recommend it to me?
answer: explanation

user_query: OK. what timeframe is it?
answer: item detail research

Which sub-task should be selected? The sub-task MUST be selected from the above list.
user_query: {query}
answer: 
'''

In [101]:
def subtask_detect(query):
    '''
    subtask detection module
    input
        query: 유저 입력문
    '''

    subtask_detection_prompt = PromptTemplate(
        template  = hj_subtask,
        input_variables = ['query']
    )

    # Chain 정의
    chain = LLMChain(
        llm = manager,
        prompt = subtask_detection_prompt
    )

    result = chain.invoke({'query':query}) # chain 실행
    return result['text']

In [121]:
subtask_type = subtask_detect('여름에 어울리는 영화 추천해줘')
print(subtask_type)

user preference elicitation


# 1) Taste Elicitation

    1. 발화에서 유저의 선호 파악하기
    2. 영화제목이라면 정확한 제목으로 mapping 하기

In [104]:
## taste elicitation

mk_scoring = """
You are a movie preference evaluator. When a user provides a preference for a specific movie, you will assess the given movie and return a JSON object that details the user's preference evaluation using English.

The answer should include the following fields:

content: Categorize the content among [director, actor, genre, movie_title]
item: A name of the content
preference_score: A numerical score indicating how much the user prefers the movie on a scale of 1 to 10.
comments: A brief explanation or comments about why the user has this preference score for the movie.

user input: 어제 에일리언을 봤는데 꽤 재밌더라구. 엄청 흥미진진해서 시간가는 줄 몰랐어
answer: 
 'content': 'movie_title'
 'item': 'Alien',
 'preference_score': 8,
 'comments': 'The user found 'Alien' to be quite entertaining and thrilling, indicating a strong engagement with the film's suspenseful elements.'

user input: 내가 가장 좋아하는 장르는 로맨스야. 절절한 사랑 이야기가 너무 좋아
answer: 
 'content':'genre',
 'item': 'romance',
 'preference_score': 9,
 'comments': 'The user loves heartfelt love stories which beautifully captures deep emotional connections and enduring love.'

user input: "신과함께 같은 영화는 너무 싫어.... 신파가 너무 많아")
 'content':'movie_title',
 'item': '신과함께',
 'preference_score': 2,
 'comments': 'The user dislikes this movie due to its excessive melodrama, which they find overwhelming.'

user input: {query}
answer: 
"""

In [109]:
# 유저의 taste elicit 하기
def taste_elicit(query):

    taste_elicitation = PromptTemplate(
        template  = mk_scoring,
        input_variables = ['query']
    )

    # Chain 정의
    chain = LLMChain(
        llm = manager,
        prompt = taste_elicitation
    )

    result = chain.invoke({'query': query}) # chain 실행

    pattern = r'^```json\s*|\s*```$'
    cleaned_text = re.sub(pattern, '', result['text'], flags=re.DOTALL).strip()
    parsed_data = json.loads(cleaned_text)

    return parsed_data

In [125]:
user_profile = {
    'movie_title': [],
    'director': [],
    'actor': [],
    'requirements': [],
    'genre': []
}

In [126]:
temp = taste_elicit('여름에 어울리는 영화 추천해줘')
user_profile[temp['content']].append((temp['item'], temp['preference_score']))
user_profile

{'movie_title': [],
 'director': [],
 'actor': [],
 'requirements': [],
 'genre': [('summer', 7)]}

# 2) Reply Bot

### 필요한 최소 정보들 목록 -> 없으면 질문하기
- 장르
- 감독
- 배우
- 선호하는 영화들 목록
- 요구사항

In [136]:
# chainlit으로 전달하기

for content in user_profile:
    if len(user_profile[content]) == 0:
        print(f'what are your favorite {content}?')


what are your favorite movie_title?
what are your favorite director?
what are your favorite actor?
what are your favorite requirements?


In [ ]:
def reply(profile):

    for k in profile:
        if len(profile[k]) == 0:
            
            # ask about it?
            # ex) what are your favorite {contents}?
    
        